<a href="https://colab.research.google.com/github/Seifeddin84/SISCOIN/blob/main/PINN_inverted_pendulum_for_human_quiet_balance_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
SIMPLIFIED PHYSICS-INFORMED NEURAL NETWORKS FOR HUMAN BALANCE
===========================================================================
> A code designed and generated by google colab gemini + claude ai

MODIFICATIONS TO FIX DAMPING ISSUE:
1. Better initialization of damping and stiffness parameters
2. Added parameter regularization loss
3. Multi-phase training to gradually enforce physics
4. Constraint damping to physiological range

REQUIRED DATA: CSV files with columns Fx, Fy, Fz, Mx, My, Mz, COPx, COPy
"""

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
import os
import re
from sklearn.metrics import mean_squared_error, r2_score

# ============================================================================
# STEP 1: SIMPLE DATA LOADER (UNCHANGED)
# ============================================================================

def load_balance_data(file_path):
    """
    Load balance data from WFDB files or CSV files.
    Returns: time, COP data, and subject info
    """

    # Try to load as WFDB first
    if os.path.exists(file_path + '.dat') and os.path.exists(file_path + '.hea'):
        print(f"Loading WFDB record: {file_path}")

        # Load the WFDB record
        record = wfdb.rdrecord(file_path)
        data = pd.DataFrame(record.p_signal, columns=record.sig_name)
        sample_rate = record.fs

        # Extract subject info from .hea file, the mass m and hieght l
        subject_info = {'weight': 75.0, 'height': 1.70}  # defaults

        hea_file = file_path + '.hea'
        if os.path.exists(hea_file):
            with open(hea_file, 'r') as f:
                content = f.read()
                # Look for height and weight in comments
                height_match = re.search(r'#Height:\s*(\d+\.?\d*)', content)
                weight_match = re.search(r'#Weight:\s*(\d+\.?\d*)', content)

                if height_match:
                    subject_info['height'] = float(height_match.group(1)) * 0.55 / 100 # convert cm to m
                if weight_match:
                    subject_info['weight'] = float(weight_match.group(1))

    else:
        # Try to load as CSV
        print(f"Loading CSV file: {file_path}")
        data = pd.read_csv(file_path)
        sample_rate = 50  # assume 100 Hz if not specified
        subject_info = {'weight': 75.0, 'height': 1.70}  # defaults for CSV

    # Make sure we have the required columns
    required_cols = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz', 'COPx', 'COPy']
    missing_cols = [col for col in required_cols if col not in data.columns]

    if missing_cols:
        raise ValueError(f"Missing columns: {missing_cols}")

    # Create time vector
    time = np.linspace(0, len(data)/sample_rate, len(data))

    # Extract COP data (what we want to predict)
    cop_data = data[['COPx', 'COPy']].values

    return time, cop_data, subject_info

# ============================================================================
# STEP 2: MODIFIED NEURAL NETWORK WITH BETTER PARAMETER INITIALIZATION
# ============================================================================

class SimpleBalancePINN(nn.Module):
    """
    A simplified neural network for balance prediction.
    MODIFIED: Better parameter initialization and constraints
    """

    def __init__(self):
        super().__init__()

        # Simple 3-layer network (unchanged)
        self.layers = nn.Sequential(
            nn.Linear(1, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 2)
        )

        # Initialize weights for better training
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)

        # MODIFIED: Better initialization for physics parameters
        # Use log parameterization to ensure positivity and better gradients
        self.log_damping = nn.Parameter(torch.tensor(np.log(5.0), dtype=torch.float32))  # exp(log(5)) = 5
        self.log_stiffness = nn.Parameter(torch.tensor(np.log(50.0), dtype=torch.float32))  # exp(log(50)) = 50

    @property
    def damping(self):
        """Always positive damping via exponential"""
        return torch.exp(self.log_damping)

    @property
    def stiffness(self):
        """Always positive stiffness via exponential"""
        return torch.exp(self.log_stiffness)

    def forward(self, t):
        """Given time t, predict COP position"""
        return self.layers(t)

# ============================================================================
# STEP 3: MODIFIED PHYSICS EQUATIONS WITH REGULARIZATION
# ============================================================================

def physics_loss(model, time_points, mass, height):
    """
    Calculate how well the model follows physics laws.
    MODIFIED: Uses the new property-based parameters
    """

    # Enable gradients for computing derivatives
    time_points.requires_grad_(True)

    # Get model predictions
    position = model(time_points)

    # Calculate velocity (first derivative)
    velocity = torch.autograd.grad(position.sum(), time_points, create_graph=True)[0]

    # Calculate acceleration (second derivative)
    acceleration = torch.autograd.grad(velocity.sum(), time_points, create_graph=True)[0]

    # MODIFIED: Use the properties (automatically positive)
    damping = model.damping
    stiffness = model.stiffness
    gravity = 9.81

    # The physics equation (unchanged)
    physics_equation = 0.01*(acceleration + (damping/(mass*height**2)) * velocity + (stiffness/(mass*height**2) - gravity/height) * position)

    # Return how much the model violates physics (smaller = better)
    return torch.mean(physics_equation**2)

# NEW: Parameter regularization function
def parameter_regularization_loss(model):
    """
    Keep parameters in physiologically reasonable ranges.
    Damping: 1-20 Nm⋅s/rad, Stiffness: 10-500 Nm/rad
    """
    damping = model.damping
    stiffness = model.stiffness

    # Soft constraints using smooth penalty functions
    damping_penalty = torch.relu(damping - 20.0)**2 + torch.relu(1.0 - damping)**2
    stiffness_penalty = torch.relu(stiffness - 500.0)**2 + torch.relu(10.0 - stiffness)**2

    return damping_penalty + stiffness_penalty

# ============================================================================
# STEP 4: MODIFIED TRAINING WITH MULTI-PHASE APPROACH
# ============================================================================

def train_simple_pinn(model, time, cop_data, subject_info, epochs=10000):
    """
    Train the neural network using both data and physics.
    MODIFIED: Multi-phase training and parameter regularization
    """

    # Convert data to PyTorch tensors
    num_data_points = min(len(time), 20000)
    t_tensor = torch.tensor(time[:num_data_points].reshape(-1, 1), dtype=torch.float32)
    cop_tensor = torch.tensor(cop_data[:num_data_points], dtype=torch.float32)

    # MODIFIED: Different optimizers for different phases
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training history
    losses = []
    damping_history = []
    stiffness_history = []

    # MODIFIED: Multi-phase training schedule
    phase_configs = [
        {'epochs': epochs//4, 'lambda_data': 1.0, 'lambda_physics': 0.1, 'lambda_reg': 0.01},
        {'epochs': epochs//4, 'lambda_data': 1.0, 'lambda_physics': 0.3, 'lambda_reg': 0.05},
        {'epochs': epochs//4, 'lambda_data': 1.0, 'lambda_physics': 0.7, 'lambda_reg': 0.1},
        {'epochs': epochs//4, 'lambda_data': 1.0, 'lambda_physics': 1.0, 'lambda_reg': 0.2}
    ]

    print(f"Starting multi-phase training with {num_data_points} data points...")

    epoch = 0
    for phase_idx, config in enumerate(phase_configs):
        print(f"\n--- Phase {phase_idx + 1}/4 ---")
        print(f"Data weight: {config['lambda_data']}, Physics weight: {config['lambda_physics']}, Regularization: {config['lambda_reg']}")

        for phase_epoch in range(config['epochs']):
            optimizer.zero_grad()

            # 1. Data loss: how well does model fit the measured data?
            predictions = model(t_tensor)
            data_loss = nn.MSELoss()(predictions, cop_tensor)

            # 2. Physics loss: how well does model follow physics?
            random_indices = torch.randint(0, num_data_points, (500, 1))
            random_times = t_tensor[random_indices]
            phys_loss = physics_loss(model, random_times, subject_info['weight'], subject_info['height'])

            # 3. MODIFIED: Add parameter regularization
            reg_loss = parameter_regularization_loss(model)

            # MODIFIED: Weighted combination based on phase
            total_loss = (config['lambda_data'] * data_loss +
                         config['lambda_physics'] * phys_loss +
                         config['lambda_reg'] * reg_loss)

            # Update the model
            total_loss.backward()

            # MODIFIED: Gradient clipping to prevent instability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Track progress
            losses.append(total_loss.item())
            damping_history.append(model.damping.item())
            stiffness_history.append(model.stiffness.item())

            # Print progress
            if epoch % 500 == 0:
                print(f"Epoch {epoch}: Total Loss = {total_loss:.6f}")
                print(f"  Damping: {model.damping.item():.4f}, Stiffness: {model.stiffness.item():.4f}")
                print(f"  Data: {data_loss.item():.6f}, Physics: {phys_loss.item():.6f}, Reg: {reg_loss.item():.6f}")

            epoch += 1

    print("Training completed!")
    return model, losses, damping_history, stiffness_history

# ============================================================================
# STEP 5: MODIFIED ANALYSIS WITH BETTER PARAMETER REPORTING
# ============================================================================

def analyze_results(model, time, cop_data, subject_info):
    """
    Analyze how well the PINN model performed with comprehensive plots.
    MODIFIED: Better parameter reporting
    """

    # Convert time to tensor for predictions
    t_tensor = torch.tensor(time.reshape(-1, 1), dtype=torch.float32)

    # Get model predictions
    model.eval()
    with torch.no_grad():
        predictions = model(t_tensor).numpy()

    # Calculate performance metrics
    true_x, true_y = cop_data[:, 0], cop_data[:, 1]
    pred_x, pred_y = predictions[:, 0], predictions[:, 1]

    rmse_x = np.sqrt(mean_squared_error(true_x, pred_x))
    rmse_y = np.sqrt(mean_squared_error(true_y, pred_y))
    r2_x = r2_score(true_x, pred_x)
    r2_y = r2_score(true_y, pred_y)

    print(f"\nPerformance Metrics:")
    print(f"RMSE X: {rmse_x:.3f}, R² X: {r2_x:.3f}")
    print(f"RMSE Y: {rmse_y:.3f}, R² Y: {r2_y:.3f}")
    print(f"\nLearned Physics Parameters:")
    print(f"Damping: {model.damping.item():.4f} Nm⋅s/rad")
    print(f"Stiffness: {model.stiffness.item():.4f} Nm/rad")

    # MODIFIED: Check if parameters are in physiological range
    damping_val = model.damping.item()
    stiffness_val = model.stiffness.item()

    damping_ok = 1.0 <= damping_val <= 20.0
    stiffness_ok = 10.0 <= stiffness_val <= 500.0

    print(f"Parameter validation:")
    print(f"  Damping {'✓' if damping_ok else '✗'} (physiological range: 1-20)")
    print(f"  Stiffness {'✓' if stiffness_ok else '✗'} (physiological range: 10-500)")

    # Create the original 2x3 plot layout (unchanged plotting code)
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    # PLOT 1: COP_x Time Series
    axes[0, 0].plot(time, true_x, 'b-', label='True', alpha=0.7, linewidth=1.5)
    axes[0, 0].plot(time, pred_x, 'r--', label='PINN', alpha=0.7, linewidth=1.5)
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('COP_x (cm)')
    axes[0, 0].legend()
    axes[0, 0].set_title(f'COP X-direction (RMSE: {rmse_x:.3f}, R²: {r2_x:.3f})')
    axes[0, 0].grid(True, alpha=0.3)

    # PLOT 2: COP_y Time Series
    axes[0, 1].plot(time, true_y, 'b-', label='True', alpha=0.7, linewidth=1.5)
    axes[0, 1].plot(time, pred_y, 'r--', label='PINN', alpha=0.7, linewidth=1.5)
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('COP_y (cm)')
    axes[0, 1].legend()
    axes[0, 1].set_title(f'COP Y-direction (RMSE: {rmse_y:.3f}, R²: {r2_y:.3f})')
    axes[0, 1].grid(True, alpha=0.3)

    # PLOT 3: Correlation Plot (True vs Predicted)
    axes[0, 2].scatter(true_x, pred_x, alpha=0.5, s=10)
    axes[0, 2].plot([true_x.min(), true_x.max()],
                    [true_x.min(), true_x.max()], 'k--', alpha=0.7)
    axes[0, 2].set_xlabel('True COP_x')
    axes[0, 2].set_ylabel('Predicted COP_x')
    axes[0, 2].set_title('Correlation COP_x')
    axes[0, 2].grid(True, alpha=0.3)

    # PLOT 4: COP Trajectory (Stabilogram)
    axes[1, 0].plot(true_x, true_y, 'b-', label='True', alpha=0.7, linewidth=1)
    axes[1, 0].plot(pred_x, pred_y, 'r--', label='PINN', alpha=0.7, linewidth=1)
    axes[1, 0].set_xlabel('COP_x (cm)')
    axes[1, 0].set_ylabel('COP_y (cm)')
    axes[1, 0].legend()
    axes[1, 0].set_title('COP Trajectory (Stabilogram)')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].axis('equal')

    # PLOT 5: Error Analysis
    error_x = true_x - pred_x
    error_y = true_y - pred_y
    axes[1, 1].plot(time, error_x, 'g-', label='Error X', alpha=0.7, linewidth=1)
    axes[1, 1].plot(time, error_y, 'm-', label='Error Y', alpha=0.7, linewidth=1)
    axes[1, 1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Error (cm)')
    axes[1, 1].legend()
    axes[1, 1].set_title('Prediction Errors')
    axes[1, 1].grid(True, alpha=0.3)

    # PLOT 6: Frequency Domain Analysis
    from scipy.fft import fft, fftfreq
    N = len(true_x)
    T = time[1] - time[0]
    freqs = fftfreq(N, T)[:N//2]

    fft_true_x = np.abs(fft(true_x))[:N//2]
    fft_pred_x = np.abs(fft(pred_x))[:N//2]

    axes[1, 2].plot(freqs[1:], fft_true_x[1:], 'b-', label='True X', alpha=0.7, linewidth=1)
    axes[1, 2].plot(freqs[1:], fft_pred_x[1:], 'r--', label='PINN X', alpha=0.7, linewidth=1)
    axes[1, 2].set_xlabel('Frequency (Hz)')
    axes[1, 2].set_ylabel('Amplitude')
    axes[1, 2].set_title('Frequency Spectrum (COP_x)')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# ============================================================================
# REMAINING FUNCTIONS UNCHANGED
# ============================================================================

def plot_physics_parameter_evolution(damping_history, stiffness_history, record_name):
    """
    Plots the evolution of learned damping and stiffness values over epochs.
    """
    epochs = range(len(damping_history))

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, damping_history, label='Learned Damping')
    plt.plot(epochs, stiffness_history, label='Learned Stiffness')

    # MODIFIED: Add physiological range indicators
    plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Damping Range (1-20)')
    plt.axhline(y=20.0, color='red', linestyle='--', alpha=0.5)
    plt.axhline(y=10.0, color='blue', linestyle='--', alpha=0.5, label='Stiffness Range (10-500)')
    plt.axhline(y=500.0, color='blue', linestyle='--', alpha=0.5)

    plt.xlabel('Epoch')
    plt.ylabel('Parameter Value')
    plt.title(f'Evolution of Learned Physics Parameters During Training ({record_name})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def run_balance_analysis(model, data_path):
    """Main function - UNCHANGED"""
    print("=== SIMPLIFIED PINN BALANCE ANALYSIS ===")

    print("\n1. Loading data...")
    time, cop_data, subject_info = load_balance_data(data_path)
    print(f"   Loaded {len(time)} data points")
    print(f"   Subject: {subject_info['weight']:.1f} kg, {subject_info['height']:.2f} m")

    print("\n2. Training PINN model...")
    model, loss_history, damping_history, stiffness_history = train_simple_pinn(model, time, cop_data, subject_info)

    print("\n3. Analyzing results...")
    analyze_results(model, time, cop_data, subject_info)

    print("\n4. Plotting physics parameter evolution...")
    record_name = os.path.basename(data_path)
    plot_physics_parameter_evolution(damping_history, stiffness_history, record_name)

    print("\nAnalysis complete!")
    return model

def analyze_multiple_files(folder_path, save_plots=True, results_folder="results"):
    """Batch processing - UNCHANGED"""
    if save_plots:
        os.makedirs(results_folder, exist_ok=True)

    dat_files = []
    for file in os.listdir(folder_path):
        if file.endswith('.dat'):
            record_name = file[:-4]
            dat_files.append(os.path.join(folder_path, record_name))

    print(f"Found {len(dat_files)} WFDB records to process")

    model = SimpleBalancePINN()
    print("\nInitialized a single PINN model for batch processing.")

    for i, record_path in enumerate(dat_files):
        print(f"\n{'='*50}")
        print(f"Processing file {i+1}/{len(dat_files)}: {os.path.basename(record_path)}")
        print(f"{'='*50}")

        try:
            model = run_balance_analysis(model, record_path)

            if save_plots:
                record_name = os.path.basename(record_path)
                plot_filename = os.path.join(results_folder, f"{record_name}_analysis.png")
                plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
                print(f"Plot saved to: {plot_filename}")

                param_plot_filename = os.path.join(results_folder, f"{record_name}_params_evolution.png")
                try:
                     plt.savefig(param_plot_filename, dpi=300, bbox_inches='tight')
                     print(f"Parameter evolution plot saved to: {param_plot_filename}")
                except Exception as save_e:
                    print(f"Error saving parameter evolution plot for {os.path.basename(record_path)}: {save_e}")

            print(f"✓ Successfully processed {os.path.basename(record_path)}")

        except Exception as e:
            print(f"✗ Error processing {os.path.basename(record_path)}: {e}")

    print(f"\n{'='*50}")
    print("All files processed!")
    if save_plots:
        print(f"Results saved to: {results_folder}")
    print(f"{'='*50}")

# ============================================================================
# MAIN LOOP - UNCHANGED
# ============================================================================

if __name__ == "__main__":
    data_folder = "/content/drive/MyDrive/human-balance-evaluation-database-1.0.0"
    analyze_multiple_files(data_folder)

"""
=== KEY MODIFICATIONS MADE TO FIX DAMPING ISSUE ===

1. **Log Parameterization**:
   - Changed from direct parameters to log_damping/log_stiffness
   - Uses properties to ensure always positive values
   - Better gradients during training

2. **Parameter Regularization**:
   - Added parameter_regularization_loss() function
   - Keeps damping in 1-20 Nm⋅s/rad range
   - Keeps stiffness in 10-500 Nm/rad range

3. **Multi-Phase Training**:
   - Gradually increases physics and regularization weights
   - Prevents premature convergence to zero damping
   - Better balance between fitting data and physics

4. **Gradient Clipping**:
   - Prevents training instability
   - Helps maintain reasonable parameter values

5. **Better Initialization**:
   - Damping starts at 5.0 (reasonable value)
   - Stiffness starts at 50.0 (reasonable value)

6. **Validation Feedback**:
   - Reports if parameters are in physiological ranges
   - Shows reference lines in parameter evolution plots

These minimal changes should prevent the damping from going to zero while
keeping the same overall structure and functionality of your original code.
"""